In [1]:
# Imports
#import FAST_RECURSION_KERNEL as frkernel  #This is the kernel that marcus is working on.
# import kernel as k
import subprocess
import io
import fetch_data as fd
import MostFrequentFeatures as mff
import svm
import numpy as np
from sklearn import metrics
import os
from tqdm import tqdm
from math import sqrt

In [ ]:
def _k_prime(s,t,n,l):

    #-------------------------------------------------------------------------------------#
    # Basically what's happening here is that we are succesively looping through both of  #
    # the strings and updating the kernel matrix accordingly while refering to previously #
    # computed values. This will give us time complexity O(n|s||t|) in the end.           #
    #-------------------------------------------------------------------------------------#
    
    
    #Variables:
    #
    #s is a string
    #t is a string
    #n is the length of the substring
    #l is the lambda value
    #kp is refering to k'
    #kpp is refering to k'' 
    
    #start by creating the empty matrices.
    kp = np.zeros([n,len(s)+1,len(t)+1]);
    kpp = np.zeros([n,len(s)+1,len(t)+1]);
    
    #initialize
    kp[0][:][:] = 1;
    
    for i in range(1,n):
        for j in range(i,len(s)):
            for k in range(i,len(t)+1):

                #check whether 'x occurs in u' as described in the paper
                if(s[j-1]!=t[k-1]):
                    kpp[i][j][k]=l*kpp[i][j][k-1];
                #if not, do the other calcs.http://localhost:8888/notebooks/To_Marcus.ipynb#
                else:
                    kpp[i][j][k]=l*(kpp[i][j][k-1]+l*kp[i-1][j-1][k-1]);
                
                #finally calculate kp
                kp[i][j][k-1]=l*kp[i][j-1][k-1]+kpp[i][j][k-1];
                
    return kp;


def _k(s,t,n,l,kp):
    
    #--------------------------------------------------#
    # This takes in an already computed k_prime kernel #
    # and calculates the overall kernel as per the     #
    # paper. The last part of Def. 2                   #
    #--------------------------------------------------#

    #Variables:
    #
    #s is a string
    #t is a string
    #n is the length of the substring
    #l is the lambda value
    #kp is refering to k'
    #ksum is refering to the kernel value.
    
    ksum = 0;
    
    #Loop over all values in the computed k_prime matrix and 
    #pick out the values where x = j, as mentioned in the paper.
    #
    #There is no recursion necessary here since we already did it
    #when computing k_prime, the last 'layer' of k_prime
    #contains all the necessary values.

    for i in range(kp.shape[1]-1):
        for j in range(kp.shape[2]-1):
            if s[i] == t[j] : 
                ksum += kp[n-1][i][j];
                
    return l**2*ksum;


In [ ]:
def approximative_kernel(x,z,s,n,l):
    N = len(x)
    kss = [ _k(i,i,n,l,_k_prime(i,i,n,l)) for i in tqdm(s)]
    kxx = [ _k(i,i,n,l,_k_prime(i,i,n,l)) for i in tqdm(x)]
    kxs = kernelValuesListChptr6(x,s,n,l)    
    if hash(tuple(x)) == hash(tuple(z)):
        K = np.identity(N)
        print('Square kernel matrix generated')
        for i,xx in enumerate(x):
            for j in range(i+1,N):
                for k,ss in enumerate(s):
                    k = (kxs[i][k]*kxs[j][k])/(kss[k]*sqrt(kxx[j]*kxx[i]))
                    K[i,j] += k
                    K[j,i] += k
        return K   

    K = np.zeros([N,len(z)])
    kzz = [ _k(i,i,n,l,_k_prime(i,i,n,l)) for i in z]
    kxz = kernelValuesListChptr6(z,s,n,l)
    for i,xx in enumerate(tqdm(x)):
        for j,zz in enumerate(z):
            for k,ss in enumerate(s):
                K[i,j] += (kxs[i][k]*kxz[j][k])/(kss[k]*sqrt(kzz[j]*kxx[i]))
    return K

def kernelValuesListChptr6(x,s,n,l):
    Kxs = np.zeros([len(x),len(s)]);
    for i in tqdm(range(len(x))):
        for j in range(len(s)):
            Kxs[i][j]=_k(x[i],s[j],n,l,_k_prime(x[i],s[j],n,l))
    return Kxs

In [2]:
### Load data
# cwd = os.getcwd()+'/../data/clean_data';
# kerns = os.path.join(cwd,'trainData.npy')
# trainData = np.load(kerns)

# kerns = os.path.join(cwd,'testData.npy')
# testData = np.load(kerns)

# kerns = os.path.join(cwd,'trainLabels.npy')
# trainLabels = np.load(kerns)

# kerns = os.path.join(cwd,'testLabels.npy')
# testLabels = np.load(kerns)

categories = ['earn','crude']
numberOfTraining = [10,10]
numberOfTesting = [7,2]


trainData,trainLabels, testData,testLabel = fd.loadData(categories,numberOfTraining,numberOfTesting)

In [3]:
#Set params
n = 4;  #how long the substrings should be.
lmda = 0.5; #The penalty (weight) paramter

In [4]:
########## OBS OBS OBS OBS OBS ################
# Det är oklart om denna kod fungerar korrekt!!!

def marcus_mostFrequentFeatures(dataset, k, numbTop,prints = False):
#     the_long_string = ''
    list_of_features = []
    for i in tqdm(dataset):
#         the_long_string += i
        the_len = len(i)
        for j in range(the_len-k+1):
            curr_str = i[j:j+n]
            if not curr_str in list_of_features:
                list_of_features.append(curr_str)
    scores = np.zeros(len(list_of_features))
    for index, feature in tqdm(enumerate(list_of_features)):
        for i in dataset:
            scores[index] += i.count(feature)
    topFeatures = []
    topFeatureScores =  []
    scores = scores.argsort()[-len(scores):][::-1]
    features = np.array(list_of_features)
    topFeatures = features[scores][:numbTop]
    topFeatureScores = scores[:numbTop]
    return topFeatures, topFeatureScores

In [5]:
#Find the S most commonly occuring n-grams

S = 500;

# topFeatures, topFeatureScores = mff.mostFrequentFeatures(trainData,n,S)
topFeatures, topFeatureScores = marcus_mostFrequentFeatures(trainData,n,S)


100%|██████████| 20/20 [00:00<00:00, 45.42it/s] 
4816it [00:00, 38302.19it/s]


In [23]:
test_list_2 = topFeatures
test_list_1 = topFeatures2
len(test_list_1)
a = 0
for i in test_list_1:
    if i in test_list_2:
        a += 1
    else:
        print(i)
print(a)

stri
ed l
498


In [9]:
#Compute kernel using these to approximate. 

Ktrain_approx = approximative_kernel(trainData,trainData,topFeatures,n,lmda)

Ktest_approx = approximative_kernel(trainData,testData,topFeatures,n,lmda)

NameError: name 'approximative_kernel' is not defined

In [7]:
# Compile the fast_apprximative_kernel
test = subprocess.Popen(["g++","fast_approximative_kernel_modified.cpp","-o","fast_approximative_kernel_modified.out"], stdout=subprocess.PIPE)
output_compile = test.communicate()[0]
# print(output_compile.decode('ascii'))
def parseMatrix(matrix):
    num_rows = int(matrix[0])
    num_cols = int(matrix[1])
    K = np.zeros([num_rows,num_cols])
    counter = 2
    for i in range(num_rows):
        for j in range(num_cols):
            K[i,j] = float(matrix[counter])
            counter = counter + 1
    return K

In [8]:
# Test fast_recursive_kernel
# x = trainData
# z = trainData
# s = topFeatures
# n = 3;  #how long the substrings should be.
# lmda = 0.5; #The penalty (weight) paramter


def marcus_approximative_kernel_modified(x,z,s,n,lmda,cutoff = 1000):
    the_strings = b''
    if hash(tuple(x)) == hash(tuple(z)):
        equal_hash = b'1 '
    else:
        equal_hash = b'0 '
    the_args = bytes(str(len(x)),'ascii')+ b' ' + bytes(str(len(z)),'ascii')+ b' ' + bytes(str(len(s)),'ascii')+ b' '
    for i in x:
        the_strings = the_strings + bytes(i,'ascii')
        the_args = the_args + bytes(str(len(i)),'ascii') + b' '
    for i in z:
        the_strings = the_strings + bytes(i,'ascii')
        the_args = the_args + bytes(str(len(i)),'ascii') + b' '
    for i in s:
        the_strings = the_strings + bytes(i,'ascii')
        the_args = the_args + bytes(str(len(i)),'ascii') + b' '
    the_args = the_args + bytes(str(n), 'ascii')+ b' ' + bytes(str(lmda), 'ascii') + b' ' + bytes(str(cutoff), 'ascii')
    fast_kernel = subprocess.Popen(["./fast_approximative_kernel_modified.out"], stdin=subprocess.PIPE, stdout=subprocess.PIPE)
    output_test = fast_kernel.communicate(input=equal_hash+the_args+the_strings)[0]
#     print(output_test.decode('utf-8'))
    output_test_list = output_test.decode('utf-8').split()
    K = parseMatrix(output_test_list)
    return K

K_approx_train_marcus = marcus_approximative_kernel_modified(trainData,trainData,topFeatures,n,lmda)
# K_approx_test_marcus = marcus_approximative_kernel_modified(trainData,testData,topFeatures,n,lmda)


In [9]:
K_approx_train_marcus

array([[ 1.        ,  0.08242561,  0.18219062,  0.20913314,  0.04293858,
         0.07750887,  0.06437695,  0.27300627,  0.19712207,  0.14814499,
         0.04310315,  0.02490148,  0.05439486,  0.06889738,  0.06388148,
         0.01877461,  0.02597555,  0.0219039 ,  0.06752055,  0.03347359],
       [ 0.08242561,  1.        ,  0.0618414 ,  0.07391461,  0.02655116,
         0.03426447,  0.04039456,  0.09169236,  0.08148054,  0.05680781,
         0.02671721,  0.01455872,  0.04785896,  0.06097451,  0.03309722,
         0.01428176,  0.02627991,  0.01108572,  0.06384466,  0.02969764],
       [ 0.18219062,  0.0618414 ,  1.        ,  0.10130453,  0.02103599,
         0.09458186,  0.05547132,  0.18488424,  0.12692223,  0.08295083,
         0.03124814,  0.0166606 ,  0.03857739,  0.05344801,  0.04079396,
         0.02456069,  0.01359084,  0.01018007,  0.05364983,  0.02882414],
       [ 0.20913314,  0.07391461,  0.10130453,  1.        ,  0.07637485,
         0.0351465 ,  0.02828235,  0.15179788,  

In [ ]:
Ktest_approx-K_approx_test_marcus